In [23]:
from __future__ import division, print_function, absolute_import
import csv
import pandas
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
import os
import xml.etree.ElementTree as etree
import cobra
import numpy as np
from itertools import chain
from cobra.util import solver as sutil
from cobra.core.solution import get_solution
from optlang.symbolics import add, Zero
import pandas as pd
import os
from os.path import join
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium
# Press ⌃R to execute it or replace it with your code.
# Press Double ⇧ to search everywhere for classes, files, tool windows, actions, and settings.
from cobra.flux_analysis import production_envelope
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import math

In [24]:
model_rs = read_sbml_model('beta_day_RS_DM_r.xml')
core_model=model_rs

In [25]:
a=[]
a=core_model.reactions.get_by_id('AraCore_Biomass_tx').reactants


In [26]:
core_model.reactions.get_by_id('AraCore_Biomass_tx').summary()


In [27]:
b=[]
a=core_model.reactions.get_by_id('AraCore_Biomass_tx').metabolites
for key in a:
    b.append(a[key])

In [28]:
c=[]
for i in a:
    #print(i)
    c.append(core_model.metabolites.get_by_id(i.id).formula_weight)

In [29]:
df=pd.DataFrame(b,dtype='float',index=list(a))
df.columns=['s']
df['for_weight']=c
df['weight']=-1*df['for_weight']*df['s']

In [30]:
biomass_factor=df['weight'].sum()

In [31]:
df['new_coeff']=df['s']/biomass_factor
met_list=df.index
coef=df['new_coeff']

In [32]:
core_model.reactions.remove('AraCore_Biomass_tx')
##
reaction = Reaction('AraCore_Biomass_tx')
reaction.name = 'Arabidopsis biomass'
reaction.lower_bound = 0.  
reaction.upper_bound = 1000.  
#Add reaction to the model
core_model.add_reactions([reaction])

In [33]:
for i in range(len(coef)):
    core_model.reactions.get_by_id('AraCore_Biomass_tx').add_metabolites({met_list[i]:coef[i]})

/var/folders/yl/47vnp0617dn9zfv19_vrxwjh0000gn/T/ipykernel_65493/2126473507.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  core_model.reactions.get_by_id('AraCore_Biomass_tx').add_metabolites({met_list[i]:coef[i]})


In [34]:
##Constraints
rubisco = core_model.problem.Constraint(3 * core_model.reactions.get_by_id("RXN_961_p").flux_expression - core_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p").flux_expression,lb=0, ub=0,)
core_model.add_cons_vars([rubisco])

atp = core_model.problem.Constraint((0.0049*core_model.reactions.get_by_id("Photon_tx").flux_expression+2.7851)-core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp)

atp_nadph_03 = core_model.problem.Constraint(3 * (core_model.reactions.get_by_id("NADPHoxm_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxc_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxp_tx").flux_expression) - core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp_nadph_03)

In [35]:
core_model.reactions.get_by_id('AraCore_Biomass_tx').summary()


In [36]:
write_sbml_model(core_model,'beta_day_RS_DM_new.xml')

In [37]:
### For Arabidopsis biomass
test_model=core_model
reaction = Reaction('Arabidopsis_biomass_tx')
reaction.name = 'Arabidopsis biomass'
reaction.lower_bound = 0.  
reaction.upper_bound = 1000.  
#Add reaction to the model
test_model.add_reactions([reaction])
##
df_new = pd.read_csv("/Users/subasrees/Desktop/core_model_test/Biomass_coefficients.csv")

#Get a list with the metabolite IDs from the generic model
met_list = []

for id in df_new.loc[:,"Ids"]:
    met = test_model.metabolites.get_by_id(id)
    met_list.append(met)

#Create a list with the coefficients
coef = list(df_new.loc[:,"Arabidopsis"])

df_new.index=df_new['Ids']
df_new.drop(columns=['Ids','Maize'],inplace=True)
df_new.columns=['s']


In [38]:
a=df_new.index
c=[]
for i in a:
    #print(i)
    c.append(test_model.metabolites.get_by_id(i).formula_weight)

In [39]:
df_new['for_weight']=c
df_new['weight']=-1*df_new['for_weight']*df_new['s']

In [40]:
biomass_coeff=df_new['weight'].sum()
df_new['new_coeff']=df_new['s']/biomass_coeff

In [41]:
test_model.objective = "Arabidopsis_biomass_tx"
test_model.reactions.remove('AraCore_Biomass_tx')
coef_new=df_new['new_coeff']
met_list_new=df_new.index
#Add metabolites to the model: dictionary - keys are metabolites and values are the coefficients
for i in range(len(coef_new)):
    test_model.reactions.get_by_id('Arabidopsis_biomass_tx').add_metabolites({met_list_new[i]:coef_new[i]})



/var/folders/yl/47vnp0617dn9zfv19_vrxwjh0000gn/T/ipykernel_65493/2971470442.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_model.reactions.get_by_id('Arabidopsis_biomass_tx').add_metabolites({met_list_new[i]:coef_new[i]})


In [42]:
#df.to_csv('/Users/subasrees/Desktop/old_biomass.csv')
#core_model=test_model
#write_sbml_model(core_model,'beta_day_RS_DM_new.xml')

In [43]:
##Constraints
rubisco = test_model.problem.Constraint(3 * test_model.reactions.get_by_id("RXN_961_p").flux_expression - test_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p").flux_expression,lb=0, ub=0,)
test_model.add_cons_vars([rubisco])

atp = test_model.problem.Constraint((0.0049*test_model.reactions.get_by_id("Photon_tx").flux_expression+2.7851)-test_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
test_model.add_cons_vars(atp)

atp_nadph_03 = test_model.problem.Constraint(3 * (test_model.reactions.get_by_id("NADPHoxm_tx").flux_expression + test_model.reactions.get_by_id("NADPHoxc_tx").flux_expression + test_model.reactions.get_by_id("NADPHoxp_tx").flux_expression) - test_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
test_model.add_cons_vars(atp_nadph_03)

In [44]:
test_model.reactions.get_by_id('Arabidopsis_biomass_tx').summary()
